In [ ]:
!pip install gdown

In [ ]:
import gdown

output = "parallel_corpus.tsv"
id = "1V7Ksb1JVxdsHDLc5QOiIeqVGQFdPf4iT"

gdown.download(id=id, output=output)

In [1]:
import math
import re
import unicodedata
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [2]:
SOS_TOKEN = 0
EOS_TOKEN = 1
PAD_TOKEN = 2

In [3]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "PAD"}
        self.n_words = 3

    def add_sentence(self, sentence):
        for word in sentence.split(" "):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


def unicode_to_ascii(s):
    return "".join(
        c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn"
    )


def normalize_string(s):
    if s is not None:
      s = unicode_to_ascii(str(s).lower().strip())
      s = re.sub(r"([.!?])", r" \1", s)
      s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
      return s.strip()
    else:
      return ""

In [4]:
def read_langs():
    print("Reading lines...")

    df = pd.read_csv('parallel_corpus.tsv', sep='\t\t\t\t\t')

    kha_lines = df['kha'].values
    en_lines = df['en'].values

    kha_lang = Lang("khasi")
    en_lang = Lang("english")

    return (
        kha_lang,
        en_lang,
        kha_lines,
        en_lines,
    )


def prepare_data():
    (
        kha_lang,
        en_lang,
        kha_lines,
        en_lines
    ) = read_langs()

    kha_lines = [normalize_string(line) for line in kha_lines]
    en_lines = [normalize_string(line) for line in en_lines]

    print("Read %s sentence pairs" % len(kha_lines))
    print("Counting words...")
    for kha_line in kha_lines:
        kha_lang.add_sentence(normalize_string(kha_line))
    for en_line in en_lines:
        en_lang.add_sentence(normalize_string(en_line))
    print("Counted words:")
    print(kha_lang.name, kha_lang.n_words)
    print(en_lang.name, en_lang.n_words)
    return (
        kha_lang,
        en_lang,
        kha_lines,
        en_lines
    )

In [5]:
class ParallelDataset(Dataset):
    def __init__(self, src_lines, tgt_lines, src_lang, tgt_lang):
        self.src_lines = src_lines
        self.tgt_lines = tgt_lines
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

    def __len__(self):
        return len(self.src_lines)

    def __getitem__(self, index):
        src_line = self.src_lines[index].strip()
        tgt_line = self.tgt_lines[index].strip()

        src_indices = [self.src_lang.word2index[word] for word in src_line.split()] + [
            EOS_TOKEN
        ]
        tgt_indices = [self.tgt_lang.word2index[word] for word in tgt_line.split()] + [
            EOS_TOKEN
        ]

        return torch.tensor(src_indices), torch.tensor(tgt_indices)

In [6]:
def pad_sequences(batch):
    # Sort the sequences by the length of the source sequence (descending)
    sorted_batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

    # Separate source and target sequences
    src_seqs = [x[0] for x in sorted_batch]
    tgt_seqs = [x[1] for x in sorted_batch]

    # Find the maximum length among both source and target sequences
    max_length = max(max(len(s) for s in src_seqs), max(len(t) for t in tgt_seqs))

    # Pad the sequences to the maximum length
    src_padded = [
        torch.cat([s, torch.tensor([PAD_TOKEN] * (max_length - len(s)))])
        for s in src_seqs
    ]
    tgt_padded = [
        torch.cat([t, torch.tensor([PAD_TOKEN] * (max_length - len(t)))])
        for t in tgt_seqs
    ]

    # Convert to tensors
    src_padded = torch.stack(src_padded)
    tgt_padded = torch.stack(tgt_padded)

    return src_padded, tgt_padded

In [7]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout: float, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(-torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("pos_embedding", pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        return self.dropout(
            token_embedding + self.pos_embedding[: token_embedding.size(0), :]
        )


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size).to(DEVICE)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


class Seq2SeqTransformer(nn.Module):
    def __init__(
        self,
        num_encoder_layers: int,
        num_decoder_layers: int,
        hidden_size: int,
        nhead: int,
        src_vocab_size: int,
        tgt_vocab_size: int,
        dim_feedforward: int = 512,
        dropout: float = 0.1,
    ):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = nn.Transformer(
            d_model=hidden_size,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.generator = nn.Linear(hidden_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, hidden_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, hidden_size)
        self.positional_encoding = PositionalEncoding(hidden_size, dropout=dropout).to(
            DEVICE
        )

    def forward(
        self,
        src: torch.Tensor,
        trg: torch.Tensor,
        src_mask: torch.Tensor,
        tgt_mask: torch.Tensor,
        src_padding_mask: torch.Tensor,
        tgt_padding_mask: torch.Tensor,
        memory_key_padding_mask: torch.Tensor,
    ):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))

        outs = self.transformer(
            src_emb,
            tgt_emb,
            src_mask,
            tgt_mask,
            None,
            src_padding_mask,
            tgt_padding_mask,
            memory_key_padding_mask,
        )
        return self.generator(outs)

In [8]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == 0).transpose(0, 1)
    tgt_padding_mask = (tgt == 0).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [9]:
if __name__ == "__main__":

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    (
        kha_lang,
        en_lang,
        kha_lines,
        en_lines,
    ) = prepare_data()

    BATCH_SIZE = 32

    # Create datasets and dataloaders
    parallel_dataset = ParallelDataset(
        kha_lines, en_lines, kha_lang, en_lang
    )

    parallel_dataloader = DataLoader(
        parallel_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_sequences
    )

    HIDDEN_SIZE = 256

    kha_vocab_size = kha_lang.n_words
    en_vocab_size = en_lang.n_words

    kha_to_en_model = Seq2SeqTransformer(
        num_encoder_layers=3,
        num_decoder_layers=3,
        hidden_size=HIDDEN_SIZE,
        nhead=8,
        src_vocab_size=kha_vocab_size,
        tgt_vocab_size=en_vocab_size,
    )

    en_to_kha_model = Seq2SeqTransformer(
        num_encoder_layers=3,
        num_decoder_layers=3,
        hidden_size=HIDDEN_SIZE,
        nhead=8,
        src_vocab_size=en_vocab_size,
        tgt_vocab_size=kha_vocab_size,
    )

    kha_to_en_model = kha_to_en_model.to(DEVICE)
    en_to_kha_model = en_to_kha_model.to(DEVICE)


    # Define optimizer
    optimizer = torch.optim.Adam(
        list(kha_to_en_model.parameters()) + list(en_to_kha_model.parameters()),
        lr=0.0001,
    )

Reading lines...


C:\Users\ahlad\AppData\Local\Temp\ipykernel_20568\2079347524.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('parallel_corpus.tsv', sep='\t\t\t\t\t')


Read 2574746 sentence pairs
Counting words...
Counted words:
khasi 491484
english 491325


c:\Users\ahlad\envs\deep-learning\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [14]:
if __name__ == "__main__":
    # Training loop
    num_epochs = 1
    for epoch in range(num_epochs):
        optimizer.zero_grad()

        # Forward pass
        src, tgt = next(iter(parallel_dataloader))

        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt)
        kha_to_en_output = kha_to_en_model(
            src, tgt, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask
        )

        en_to_kha_output = en_to_kha_model(
            tgt, src, tgt_mask, src_mask, tgt_padding_mask, src_padding_mask, tgt_padding_mask
        )

        # Calculate loss
        loss_kha_to_en = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)(
            kha_to_en_output.reshape(-1, kha_vocab_size), tgt.reshape(-1)
        )

        loss_en_to_kha = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)(
            en_to_kha_output.reshape(-1, en_vocab_size), src.reshape(-1)
        )

        loss = loss_kha_to_en + loss_en_to_kha
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print loss for monitoring
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}")

        # Save the trained models
        torch.save(kha_to_en_model.state_dict(), "kha_to_en_model.pth")
        torch.save(en_to_kha_model.state_dict(), "en_to_kha_model.pth")

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.34 GiB. GPU 